# Group 3

Dave Carlson, Brooke, Daniel

## Imports and set up LLM client

In [2]:
import sys
import os
import json

from IPython.display import display, Markdown, Code

try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils import setup_llm_client, prompt_enhancer, get_completion

llm_model = "gpt-4o"

client, model_name, api_provider = setup_llm_client(llm_model)

def display_markdown(text):
    display(Markdown(text))

def display_code(code, language='json'):
    display(Code(code, language=language))

write_artifacts = False

2025-10-02 00:22:41,459 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Problem Statement

In [3]:
problem_statement = """
I am seeking a web application that can help me manage my personal finances more effectively. 
- The application should allow me to track my income and expenses, create budgets, and generate reports to visualize my spending patterns. 
- Additionally, I would like the app to provide reminders for upcoming bills and suggest ways to save money based on my spending habits. 
- The interface should be user-friendly and accessible from both desktop and mobile devices.
"""

### Enhanced Problem Statement

In [4]:
enhanced_problem_statement = prompt_enhancer(problem_statement)
display_markdown(enhanced_problem_statement)

2025-10-02 00:22:49,219 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<prompt>
    <persona>
        You are a Senior FinTech Product Consultant with 10+ years of experience evaluating and implementing personal-finance solutions for consumers and small businesses.
    </persona>

    <context>
        A user is searching for a web-based application to manage personal finances more effectively.  
        Mandatory capabilities:  
        • Track income and expenses  
        • Create and monitor budgets  
        • Generate visual reports of spending patterns  
        • Provide reminders for upcoming bills  
        • Suggest money-saving opportunities derived from the user’s spending habits  
        • Offer a user-friendly interface accessible on both desktop and mobile devices  

        Additional constraints:  
        • The user prefers solutions that are easy to set up and maintain.  
        • Budget for the application should be clearly stated if pricing is involved.  
        • Recommendations must be unbiased and based on up-to-date market information (≤ 6 months old).
    </context>

    <instructions>
        1. Think step-by-step to identify and screen current personal-finance web applications.  
        2. Select the 3–5 best-fit products that satisfy (or closely approximate) all mandatory capabilities.  
        3. For each product, analyze feature coverage, usability, pricing, and platform support.  
        4. Highlight any notable trade-offs, gaps, or unique advantages.  
        5. Conclude with a clear, actionable recommendation for the user, including optional next steps if no single product meets 100 % of requirements.
    </instructions>

    <output_format>
        Produce a Markdown report with the following structure:

        ## Executive Summary  
        • 2–3 sentences summarizing key findings.

        ## Comparison Table  
        | Application | Income/Expense Tracking | Budgeting | Reports & Visuals | Bill Reminders | Savings Suggestions | Platforms (Web/iOS/Android) | Pricing (Free/Paid) | Pros | Cons |

        ## Detailed Analysis  
        ### {Application 1}  
        – Bullet-point evaluation (100–120 words)  

        ### {Application 2}  
        – …

        *(Repeat for each application.)*

        ## Final Recommendation  
        • 1–2 paragraphs (150–200 words) stating the best option(s) and rationale, plus any suggested next actions.
    </output_format>
</prompt>

## Brainstorm Features

In [5]:
# Set the output directory based on API provider and model
output_dir = f"../artifacts/documents/{api_provider}/{llm_model}"
os.makedirs(output_dir, exist_ok=True)

# Brainstorm features prompt
brainstorm_prompt = f"""
Based on the problem statement, brainstorm a list of potential features that could address the problem.
- Keep the list concise and focused on high-impact features.
- Provide a brief description for each feature.
- Format the output in a markdown list format without the ```markdown``` fences.

<Problem Statement>
{enhanced_problem_statement}
</Problem Statement>
"""

brainstormed_features = get_completion(brainstorm_prompt, client, model_name, api_provider)
display(Markdown(brainstormed_features))

if write_artifacts:
    with open(os.path.join(output_dir, "brainstormed_features.md"), "w") as f:
        f.write(brainstormed_features)

## Executive Summary  
• The analysis identified three top personal-finance applications that meet most mandatory capabilities: Mint, YNAB (You Need A Budget), and PocketGuard. Each offers a unique combination of features for tracking finances and managing budgets effectively. While all applications support multiple platforms, they differ in pricing and specific functionalities.

## Comparison Table  
| Application | Income/Expense Tracking | Budgeting | Reports & Visuals | Bill Reminders | Savings Suggestions | Platforms (Web/iOS/Android) | Pricing (Free/Paid) | Pros | Cons |
|-------------|-------------------------|-----------|-------------------|----------------|---------------------|-----------------------------|---------------------|------|------|
| Mint        | Yes                     | Yes       | Yes               | Yes            | Yes                 | Web, iOS, Android           | Free                | Comprehensive free features | Ads and third-party offers |
| YNAB        | Yes                     | Yes       | Yes               | No             | Yes                 | Web, iOS, Android           | Paid                | Great for proactive budgeting | Higher learning curve |
| PocketGuard | Yes                     | Yes       | Yes               | Yes            | Yes                 | Web, iOS, Android           | Free/Paid           | Easy setup and use | Limited advanced features in free version |

## Detailed Analysis  
### Mint  
– Mint offers robust tools for tracking income and expenses, creating budgets, and viewing spending reports. Its intuitive interface is accessible on all major platforms. While free, it incorporates advertising and third-party offers, which some users may find intrusive. Mint excels in providing a comprehensive overview of finances and automatic bill reminders, making it ideal for users seeking a free, all-in-one solution.

### YNAB  
– YNAB is designed for proactive budgeters, focusing on allocating every dollar a job. It supports detailed expense tracking and insightful reports but lacks built-in bill reminders. Available on multiple platforms, it requires a paid subscription, reflecting its advanced budget management capabilities. YNAB is highly recommended for users willing to invest time and resources in mastering its system for long-term financial discipline.

### PocketGuard  
– PocketGuard simplifies personal finance management with straightforward income and expense tracking, budgeting, and bill reminders. It provides savings suggestions based on user data and is accessible on all major platforms. The free version is easy to set up and maintain, though it lacks some advanced features found in competitors. Its premium version offers expanded tools for users seeking more in-depth financial insights.

## Final Recommendation  
• For users prioritizing a comprehensive yet free solution, Mint is the best option, offering a wide range of features with minimal setup. It is ideal for those comfortable with advertisements and seeking a full suite of financial tools. For users focused on budgeting and willing to pay for advanced capabilities, YNAB is highly recommended due to its structured approach and insightful spending analysis. PocketGuard is suitable for users who prefer simplicity and ease of use, with the option to upgrade for additional features. If no single application fully meets all needs, combining Mint or PocketGuard with a dedicated bill reminder app could offer an effective workaround.

## Generate User Personas

In [6]:
personas_prompt = f"""
<Prompt Instructions>
Based on the problem statement and the brainstormed features, identify at least three, but no more than five, unique user personas who would benefit from this tool. 
- The roles and their experience should vary between each persona. 
- Include a mix of technical and non-technical users.
- Provide the following for each persona:
  - Name
  - Role
  - Experience Level
  - Goals
  - Challenges
  - How this tool can help
- Format the output in a markdown list format without the ```markdown``` fences.
</Prompt Instructions>

<Problem Statement>   
{enhanced_problem_statement}
</Problem Statement>

<Brainstormed Features>
{brainstormed_features}
</Brainstormed Features>"""

user_personas = get_completion(personas_prompt, client, model_name, api_provider)
display_markdown(user_personas)

if write_artifacts:
    with open(os.path.join(output_dir, "user_personas.md"), "w") as f:
        f.write(user_personas)

- **Name:** Alex Turner  
  - **Role:** College Student  
  - **Experience Level:** Beginner  
  - **Goals:** Alex aims to manage his limited budget effectively, ensuring he can track his expenses, save for future needs, and avoid any late payment fees for bills.  
  - **Challenges:** Limited financial literacy and finding time to manage finances between classes and part-time work.  
  - **How this tool can help:** An easy-to-use application like Mint can offer Alex a free, comprehensive overview of his finances, track his spending, provide bill reminders, and suggest ways to save money without overwhelming him with complex features.

- **Name:** Sarah Johnson  
  - **Role:** Small Business Owner  
  - **Experience Level:** Intermediate  
  - **Goals:** Sarah wants to streamline her personal and business finances, ensuring clear visibility over income and expenses, and effective budgeting to enhance savings and business growth.  
  - **Challenges:** Balancing time between running her business and managing personal finances, and ensuring accurate financial tracking.  
  - **How this tool can help:** YNAB's structured budgeting approach can help Sarah allocate funds efficiently, while its detailed reports provide insights that can translate into better financial decisions for both personal and business purposes.

- **Name:** Mark Chen  
  - **Role:** Working Professional  
  - **Experience Level:** Advanced  
  - **Goals:** Mark seeks to optimize his financial strategy, aiming to maximize savings and investments while ensuring all expenses are tracked and within budget.  
  - **Challenges:** Managing a complex financial portfolio with limited time due to a demanding job.  
  - **How this tool can help:** PocketGuard's ease of use and savings suggestions can assist Mark in maintaining financial discipline without extensive time investment, with the option to upgrade for deeper insights if needed.

- **Name:** Emily Rivera  
  - **Role:** Retiree  
  - **Experience Level:** Beginner  
  - **Goals:** Emily wants to ensure her retirement funds are managed properly, track her spending, and get reminders for recurring bills to prevent any lapses.  
  - **Challenges:** Limited experience with digital tools and managing a fixed income.  
  - **How this tool can help:** Mint, with its user-friendly interface and comprehensive features, can help Emily keep track of her finances without needing technical expertise, providing peace of mind with automatic bill reminders and visual spending reports.

- **Name:** David Singh  
  - **Role:** Freelance Designer  
  - **Experience Level:** Intermediate  
  - **Goals:** David aims to track irregular income, manage expenses efficiently, and save up for future projects and personal goals.  
  - **Challenges:** Irregular income patterns and the need for a flexible budgeting tool that adapts to varying financial situations.  
  - **How this tool can help:** YNAB's focus on proactive budgeting can assist David in planning for irregular income and ensuring every dollar is used effectively, helping him stay on top of his financial goals despite the variability of his earnings.

## Generate User Stories

In [ ]:
personas_prompt = f"""
<Prompt Instructions>
You are a senior product manager.
- Based on the problem statement, brainstormed features, and user personas, create at least three but no more than five user stories per persona.
- Your output should be a properly-structured JSON with no markdown fences.
- Each user story should contain the following keys:
    - "ID": A unique identifier for the user story in a two-digit format with a leading zero.
    - "persona": The persona this user story is for.
    - "user_story": A brief description of the user story.
    - "acceptance_criteria": A list of at least three acceptance criteria for the user story, written in Gherkin format i.e., given/when/then statements.
- Ensure the output is only valid JSON code and can be parsed without errors.
</Prompt Instructions>

<Problem Statement>
{enhanced_problem_statement}
</Problem Statement>

<Brainstormed Features>
{brainstormed_features}
</Brainstormed Features>

<User Personas>
{user_personas}
</User Personas>"""

print("Generating user stories...")
user_stories = get_completion(personas_prompt, client, model_name, api_provider)

try:
    if "```" in user_stories:
        user_stories = user_stories.split("```")[1].lstrip("json").strip()
        user_stories = json.loads(user_stories)
        print("Successfully parsed user stories JSON.")

    user_stories = json.loads(user_stories)
    print(json.dumps(user_stories, indent=4))
    if write_artifacts:
        with open(os.path.join(output_dir, "user_stories.json"), "w") as f:
            json.dump(user_stories, f, indent=4)

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. \nError: {e}")
    print(f"LLM Output:\n{user_stories}")
    user_stories = []


Generating user stories...
{
    "user_stories": [
        {
            "ID": "01",
            "persona": "Alex Turner",
            "user_story": "As a college student, I want to receive reminders for upcoming bills so that I can avoid late payment fees.",
            "acceptance_criteria": [
                "Given I have upcoming bills, when the due date is approaching, then I receive a reminder notification.",
                "Given I set a bill reminder, when I check my notifications, then I see a list of upcoming bills.",
                "Given a bill is due soon, when I receive a notification, then it includes the bill amount and due date."
            ]
        },
        {
            "ID": "02",
            "persona": "Alex Turner",
            "user_story": "As a college student, I want to track my daily expenses easily to manage my limited budget.",
            "acceptance_criteria": [
                "Given I log an expense, when I submit the entry, then it is saved in my

In [22]:
print(json.dumps(json.loads(user_stories), indent=4))

{
    "user_stories": [
        {
            "ID": "01",
            "persona": "Alex Turner",
            "user_story": "As a college student, I want to receive reminders for upcoming bill payments so that I can avoid late fees.",
            "acceptance_criteria": [
                "Given I have upcoming bills, when the bill's due date approaches, then I should receive a notification reminder.",
                "Given I set up a bill reminder, when the bill is due within 7 days, then I should receive a push notification on my phone.",
                "Given I have multiple bills, when I view my dashboard, then I should see a list of all upcoming bills with due dates."
            ]
        },
        {
            "ID": "02",
            "persona": "Alex Turner",
            "user_story": "As a college student, I want to see visual reports of my spending patterns so that I can easily understand where my money goes.",
            "acceptance_criteria": [
                "Given I have